In [289]:
import pandas as pd

## Importing datasets

In [290]:
bios = pd.read_csv('./data/bios.csv')
results = pd.read_csv('./data/results.csv')
bios.head()

C:\Users\danie\AppData\Local\Temp\ipykernel_18304\2861276808.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  results = pd.read_csv('./data/results.csv')


,Roles,Sex,Full name,Used name,Born,Died,NOC,athlete_id,Measurements,Affiliations,Nick/petnames,Title(s),Other names,Nationality,Original name,Name order
0,Competed in Olympic Games,Male,"François Joseph Marie Antoine ""Jean-François""•...",Jean-François•Blanchy,"12 December 1886 in Bordeaux, Gironde (FRA)","2 October 1960 in Saint-Jean-de-Luz, Pyrénées-...",France,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Competed in Olympic Games,Male,Arnaud Benjamin•Boetsch,Arnaud•Boetsch,"1 April 1969 in Meulan, Yvelines (FRA)",NaN,France,2,183 cm / 76 kg,"Racing Club de France, Paris (FRA)",NaN,NaN,NaN,NaN,NaN,NaN
2,Competed in Olympic Games • Administrator,Male,Jean Laurent Robert•Borotra,Jean•Borotra,"13 August 1898 in Biarritz, Pyrénées-Atlantiqu...","17 July 1994 in Arbonne, Pyrénées-Atlantiques ...",France,3,183 cm / 76 kg,"TCP, Paris (FRA)",Le Basque Bondissant (The Bounding Basque),NaN,NaN,NaN,NaN,NaN
3,Competed in Olympic Games,Male,Jacques Marie Stanislas Jean•Brugnon,Jacques•Brugnon,"11 May 1895 in Paris VIIIe, Paris (FRA)","20 March 1978 in Monaco, Monaco (MON)",France,4,168 cm / 64 kg,"Sporting club de Paris, Paris (FRA)",Toto,NaN,NaN,NaN,NaN,NaN
4,Competed in Olympic Games,Male,Henry Albert•Canet,Albert•Canet,"17 April 1878 in Wandsworth, England (GBR)","25 July 1930 in Paris VIIe, Paris (FRA)",France,5,NaN,"TCP, Paris (FRA)",NaN,NaN,NaN,NaN,NaN,NaN


### What should I clean up?

- [x] Get rid of extra columns
- [x] Get rid of bullet points in used names
- [x] Split height and weight 
- [x] Parse out the dates in `born` and `died` columns
- [ ] Parse out country, region and city in `born` column

In [291]:
df = bios.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145500 entries, 0 to 145499
Data columns (total 16 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Roles          145500 non-null  object
 1   Sex            145500 non-null  object
 2   Full name      145500 non-null  object
 3   Used name      145500 non-null  object
 4   Born           143772 non-null  object
 5   Died           34042 non-null   object
 6   NOC            145499 non-null  object
 7   athlete_id     145500 non-null  int64 
 8   Measurements   107833 non-null  object
 9   Affiliations   95832 non-null   object
 10  Nick/petnames  9145 non-null    object
 11  Title(s)       399 non-null     object
 12  Other names    7167 non-null    object
 13  Nationality    8259 non-null    object
 14  Original name  30739 non-null   object
 15  Name order     7844 non-null    object
dtypes: int64(1), object(15)
memory usage: 17.8+ MB


In [292]:
df['Affiliations'].notna().sum()

np.int64(95832)

In [293]:
def removing_columns(df):
    for column in df.columns:
        if df[column].notna().sum() < 100000:
            df = df.drop(columns=[column])
    return df
df_nreduced = removing_columns(df)
df_nreduced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145500 entries, 0 to 145499
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   Roles         145500 non-null  object
 1   Sex           145500 non-null  object
 2   Full name     145500 non-null  object
 3   Used name     145500 non-null  object
 4   Born          143772 non-null  object
 5   NOC           145499 non-null  object
 6   athlete_id    145500 non-null  int64 
 7   Measurements  107833 non-null  object
dtypes: int64(1), object(7)
memory usage: 8.9+ MB


In [294]:
df_nreduced.head()

,Roles,Sex,Full name,Used name,Born,NOC,athlete_id,Measurements
0,Competed in Olympic Games,Male,"François Joseph Marie Antoine ""Jean-François""•...",Jean-François•Blanchy,"12 December 1886 in Bordeaux, Gironde (FRA)",France,1,NaN
1,Competed in Olympic Games,Male,Arnaud Benjamin•Boetsch,Arnaud•Boetsch,"1 April 1969 in Meulan, Yvelines (FRA)",France,2,183 cm / 76 kg
2,Competed in Olympic Games • Administrator,Male,Jean Laurent Robert•Borotra,Jean•Borotra,"13 August 1898 in Biarritz, Pyrénées-Atlantiqu...",France,3,183 cm / 76 kg
3,Competed in Olympic Games,Male,Jacques Marie Stanislas Jean•Brugnon,Jacques•Brugnon,"11 May 1895 in Paris VIIIe, Paris (FRA)",France,4,168 cm / 64 kg
4,Competed in Olympic Games,Male,Henry Albert•Canet,Albert•Canet,"17 April 1878 in Wandsworth, England (GBR)",France,5,NaN


In [295]:
df_nreduced['Used name'] = df_nreduced['Used name'].str.replace('•', ' ', regex=False)
df_nreduced

,Roles,Sex,Full name,Used name,Born,NOC,athlete_id,Measurements
0,Competed in Olympic Games,Male,"François Joseph Marie Antoine ""Jean-François""•...",Jean-François Blanchy,"12 December 1886 in Bordeaux, Gironde (FRA)",France,1,NaN
1,Competed in Olympic Games,Male,Arnaud Benjamin•Boetsch,Arnaud Boetsch,"1 April 1969 in Meulan, Yvelines (FRA)",France,2,183 cm / 76 kg
2,Competed in Olympic Games • Administrator,Male,Jean Laurent Robert•Borotra,Jean Borotra,"13 August 1898 in Biarritz, Pyrénées-Atlantiqu...",France,3,183 cm / 76 kg
3,Competed in Olympic Games,Male,Jacques Marie Stanislas Jean•Brugnon,Jacques Brugnon,"11 May 1895 in Paris VIIIe, Paris (FRA)",France,4,168 cm / 64 kg
4,Competed in Olympic Games,Male,Henry Albert•Canet,Albert Canet,"17 April 1878 in Wandsworth, England (GBR)",France,5,NaN
...,...,...,...,...,...,...,...,...
145495,Competed in Olympic Games,Female,Polina•Luchnikova,Polina Luchnikova,"30 January 2002 in Serov, Sverdlovsk (RUS)",ROC,149222,167 cm / 61 kg
145496,Competed in Olympic Games,Female,Valeriya Sergeyevna•Merkusheva,Valeriya Merkusheva,"20 September 1999 in Moskva (Moscow), Moskva (...",ROC,149223,168 cm / 65 kg
145497,Competed in Olympic Games,Female,Yuliya•Smirnova,Yuliya Smirnova,"8 May 1998 in Kotlas, Arkhangelsk (RUS)",ROC,149224,163 cm / 55 kg
145498,Competed in Olympic Games,Male,André•Foussard,André Foussard,"19 May 1899 in Niort, Deux-Sèvres (FRA)",France,149225,166 cm


In [296]:
df_nreduced.drop(columns = 'Full name', inplace=True)
df_nreduced.head()

,Roles,Sex,Used name,Born,NOC,athlete_id,Measurements
0,Competed in Olympic Games,Male,Jean-François Blanchy,"12 December 1886 in Bordeaux, Gironde (FRA)",France,1,NaN
1,Competed in Olympic Games,Male,Arnaud Boetsch,"1 April 1969 in Meulan, Yvelines (FRA)",France,2,183 cm / 76 kg
2,Competed in Olympic Games • Administrator,Male,Jean Borotra,"13 August 1898 in Biarritz, Pyrénées-Atlantiqu...",France,3,183 cm / 76 kg
3,Competed in Olympic Games,Male,Jacques Brugnon,"11 May 1895 in Paris VIIIe, Paris (FRA)",France,4,168 cm / 64 kg
4,Competed in Olympic Games,Male,Albert Canet,"17 April 1878 in Wandsworth, England (GBR)",France,5,NaN


In [297]:
# Splitting 'Measurements' into 'height_cm' and 'weight_kg'
df_nreduced[['height_cm', 'weight_kg']] = df_nreduced['Measurements'].str.split('/', expand=True)
df_nreduced.head()

,Roles,Sex,Used name,Born,NOC,athlete_id,Measurements,height_cm,weight_kg
0,Competed in Olympic Games,Male,Jean-François Blanchy,"12 December 1886 in Bordeaux, Gironde (FRA)",France,1,NaN,NaN,NaN
1,Competed in Olympic Games,Male,Arnaud Boetsch,"1 April 1969 in Meulan, Yvelines (FRA)",France,2,183 cm / 76 kg,183 cm,76 kg
2,Competed in Olympic Games • Administrator,Male,Jean Borotra,"13 August 1898 in Biarritz, Pyrénées-Atlantiqu...",France,3,183 cm / 76 kg,183 cm,76 kg
3,Competed in Olympic Games,Male,Jacques Brugnon,"11 May 1895 in Paris VIIIe, Paris (FRA)",France,4,168 cm / 64 kg,168 cm,64 kg
4,Competed in Olympic Games,Male,Albert Canet,"17 April 1878 in Wandsworth, England (GBR)",France,5,NaN,NaN,NaN


In [298]:
# Cleaning 'Measurements' column to keep only valid entries
df_nreduced = df_nreduced[(df_nreduced['Measurements'].str.contains('/', na=False) & df_nreduced['Measurements'].notna())] 
df_nreduced

,Roles,Sex,Used name,Born,NOC,athlete_id,Measurements,height_cm,weight_kg
1,Competed in Olympic Games,Male,Arnaud Boetsch,"1 April 1969 in Meulan, Yvelines (FRA)",France,2,183 cm / 76 kg,183 cm,76 kg
2,Competed in Olympic Games • Administrator,Male,Jean Borotra,"13 August 1898 in Biarritz, Pyrénées-Atlantiqu...",France,3,183 cm / 76 kg,183 cm,76 kg
3,Competed in Olympic Games,Male,Jacques Brugnon,"11 May 1895 in Paris VIIIe, Paris (FRA)",France,4,168 cm / 64 kg,168 cm,64 kg
5,Competed in Olympic Games,Male,Nicolas Chatelain,"13 January 1970 in Amiens, Somme (FRA)",France,6,181 cm / 70 kg,181 cm,70 kg
6,Competed in Olympic Games,Male,Patrick Chila,"27 November 1969 in Ris-Orangis, Essonne (FRA)",France,7,180 cm / 73 kg,180 cm,73 kg
...,...,...,...,...,...,...,...,...,...
145473,Competed in Olympic Games,Male,Toms Andersons,"25 November 1993 in Rīga, Rīga (LAT)",Latvia,149200,185 cm / 86 kg,185 cm,86 kg
145474,Competed in Olympic Games,Female,Nadine Hofstetter,"21 October 1994 in Romoos, Luzern (SUI)",Switzerland,149201,164 cm / 68 kg,164 cm,68 kg
145495,Competed in Olympic Games,Female,Polina Luchnikova,"30 January 2002 in Serov, Sverdlovsk (RUS)",ROC,149222,167 cm / 61 kg,167 cm,61 kg
145496,Competed in Olympic Games,Female,Valeriya Merkusheva,"20 September 1999 in Moskva (Moscow), Moskva (...",ROC,149223,168 cm / 65 kg,168 cm,65 kg


In [299]:
# Handling weight ranges in 'weight_kg' column
df_nreduced['weight_kg'] = df_nreduced['weight_kg'].str.replace('kg', '', regex=False)

## Changing weight ranges to their average
mask_kg = df_nreduced['weight_kg'].str.contains('-', na=False)
df_nreduced.loc[mask_kg, 'weight_kg'] = (df_nreduced.loc[mask_kg, 'weight_kg'].str.split('-').str[0].astype(float) + df_nreduced.loc[mask_kg, 'weight_kg'].str.split('-').str[1].astype(float))/2


df_nreduced.head()

C:\Users\danie\AppData\Local\Temp\ipykernel_18304\2770255200.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nreduced['weight_kg'] = df_nreduced['weight_kg'].str.replace('kg', '', regex=False)


,Roles,Sex,Used name,Born,NOC,athlete_id,Measurements,height_cm,weight_kg
1,Competed in Olympic Games,Male,Arnaud Boetsch,"1 April 1969 in Meulan, Yvelines (FRA)",France,2,183 cm / 76 kg,183 cm,76
2,Competed in Olympic Games • Administrator,Male,Jean Borotra,"13 August 1898 in Biarritz, Pyrénées-Atlantiqu...",France,3,183 cm / 76 kg,183 cm,76
3,Competed in Olympic Games,Male,Jacques Brugnon,"11 May 1895 in Paris VIIIe, Paris (FRA)",France,4,168 cm / 64 kg,168 cm,64
5,Competed in Olympic Games,Male,Nicolas Chatelain,"13 January 1970 in Amiens, Somme (FRA)",France,6,181 cm / 70 kg,181 cm,70
6,Competed in Olympic Games,Male,Patrick Chila,"27 November 1969 in Ris-Orangis, Essonne (FRA)",France,7,180 cm / 73 kg,180 cm,73


In [300]:
# Handling weight ranges in 'height_cm' column
df_nreduced['height_cm'] = df_nreduced['height_cm'].str.replace('cm', '', regex=False).astype(float)
df_nreduced.head(10)

C:\Users\danie\AppData\Local\Temp\ipykernel_18304\1337999705.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nreduced['height_cm'] = df_nreduced['height_cm'].str.replace('cm', '', regex=False).astype(float)


,Roles,Sex,Used name,Born,NOC,athlete_id,Measurements,height_cm,weight_kg
1,Competed in Olympic Games,Male,Arnaud Boetsch,"1 April 1969 in Meulan, Yvelines (FRA)",France,2,183 cm / 76 kg,183.0,76
2,Competed in Olympic Games • Administrator,Male,Jean Borotra,"13 August 1898 in Biarritz, Pyrénées-Atlantiqu...",France,3,183 cm / 76 kg,183.0,76
3,Competed in Olympic Games,Male,Jacques Brugnon,"11 May 1895 in Paris VIIIe, Paris (FRA)",France,4,168 cm / 64 kg,168.0,64
5,Competed in Olympic Games,Male,Nicolas Chatelain,"13 January 1970 in Amiens, Somme (FRA)",France,6,181 cm / 70 kg,181.0,70
6,Competed in Olympic Games,Male,Patrick Chila,"27 November 1969 in Ris-Orangis, Essonne (FRA)",France,7,180 cm / 73 kg,180.0,73
14,Competed in Olympic Games,Male,Damien Éloi,"4 July 1969 in Vire, Calvados (FRA)",France,15,165 cm / 58 kg,165.0,58
16,Competed in Olympic Games • Competed in Olympi...,Male,Guy Forget,"4 January 1965 in Casablanca, Casablanca-Setta...",France,17,189 cm / 79 kg,189.0,79
17,Competed in Olympic Games,Male,Jean-Philippe Gatien,"16 October 1968 in Alès, Gard (FRA)",France,18,178 cm / 73 kg,178.0,73
25,Competed in Olympic Games,Male,Henri Leconte,"4 July 1963 in Lillers, Pas-de-Calais (FRA)",France,26,184 cm / 78 kg,184.0,78
26,Competed in Olympic Games,Male,Christophe Legoût,"6 August 1973 in Montbéliard, Doubs (FRA)",France,27,177 cm / 75 kg,177.0,75


In [301]:
df_nreduced.sort_values(by='height_cm', ascending=False)

,Roles,Sex,Used name,Born,NOC,athlete_id,Measurements,height_cm,weight_kg
89070,Competed in Olympic Games • Other,Male,Yao Ming,"12 September 1980 in Xuhui District, Shanghai ...",People's Republic of China,89782,226 cm / 141 kg,226.0,141
5781,Competed in Olympic Games,Male,Tommy Burleson,"24 February 1952 in Crossnore, North Carolina ...",United States,5804,223 cm / 102 kg,223.0,102
6978,Competed in Olympic Games,Male,Arvydas Sabonis,"19 December 1964 in Kaunas, Kaunas (LTU)",Lithuania Soviet Union,7013,223 cm / 122 kg,223.0,122
120266,Competed in Olympic Games,Male,Zhang Zhaoxu,"18 November 1987 in Binzhou, Shandong (CHN)",People's Republic of China,122147,221 cm / 110 kg,221.0,110
89075,Competed in Olympic Games,Male,Roberto Dueñas,"1 November 1975 in Madrid, Madrid (ESP)",Spain,89787,221 cm / 137 kg,221.0,137
...,...,...,...,...,...,...,...,...,...
4517,Competed in Olympic Games,Male,Boston Simbeye,24 December 1959,Malawi,4532,130 cm / 67 kg,130.0,67
4664,Competed in Olympic Games,Male,Salvador Miranda,13 September 1949,Nicaragua,4681,130 cm / 51 kg,130.0,51
4516,Competed in Olympic Games,Male,Helman Palije,24 June 1967 in ? (MAW),Malawi,4531,128 cm / 74 kg,128.0,74
4515,Competed in Olympic Games,Male,Lyton Mphande,14 May 1963 in ? (MAW),Malawi,4530,127 cm / 62 kg,127.0,62


In [302]:
# Parse out dates from 'Born' column
### Using regular expressions to extract year, month, day 
date_pattern = r'(\d+ \w+ \d{4}|\d{4})'
df_nreduced['born_date'] = df_nreduced['Born'].str.extract(date_pattern) # \d+ = digit, \w+ = word or character, \d{4} = 4 digits
df_nreduced.head()

C:\Users\danie\AppData\Local\Temp\ipykernel_18304\11372016.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nreduced['born_date'] = df_nreduced['Born'].str.extract(date_pattern) # \d+ = digit, \w+ = word or character, \d{4} = 4 digits


,Roles,Sex,Used name,Born,NOC,athlete_id,Measurements,height_cm,weight_kg,born_date
1,Competed in Olympic Games,Male,Arnaud Boetsch,"1 April 1969 in Meulan, Yvelines (FRA)",France,2,183 cm / 76 kg,183.0,76,1 April 1969
2,Competed in Olympic Games • Administrator,Male,Jean Borotra,"13 August 1898 in Biarritz, Pyrénées-Atlantiqu...",France,3,183 cm / 76 kg,183.0,76,13 August 1898
3,Competed in Olympic Games,Male,Jacques Brugnon,"11 May 1895 in Paris VIIIe, Paris (FRA)",France,4,168 cm / 64 kg,168.0,64,11 May 1895
5,Competed in Olympic Games,Male,Nicolas Chatelain,"13 January 1970 in Amiens, Somme (FRA)",France,6,181 cm / 70 kg,181.0,70,13 January 1970
6,Competed in Olympic Games,Male,Patrick Chila,"27 November 1969 in Ris-Orangis, Essonne (FRA)",France,7,180 cm / 73 kg,180.0,73,27 November 1969


In [303]:
df_nreduced['born_date'] = pd.to_datetime(df_nreduced['born_date'], errors='coerce')

C:\Users\danie\AppData\Local\Temp\ipykernel_18304\1631298919.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nreduced['born_date'] = pd.to_datetime(df_nreduced['born_date'], errors='coerce')


In [304]:
df_nreduced.head()

,Roles,Sex,Used name,Born,NOC,athlete_id,Measurements,height_cm,weight_kg,born_date
1,Competed in Olympic Games,Male,Arnaud Boetsch,"1 April 1969 in Meulan, Yvelines (FRA)",France,2,183 cm / 76 kg,183.0,76,1969-04-01
2,Competed in Olympic Games • Administrator,Male,Jean Borotra,"13 August 1898 in Biarritz, Pyrénées-Atlantiqu...",France,3,183 cm / 76 kg,183.0,76,1898-08-13
3,Competed in Olympic Games,Male,Jacques Brugnon,"11 May 1895 in Paris VIIIe, Paris (FRA)",France,4,168 cm / 64 kg,168.0,64,1895-05-11
5,Competed in Olympic Games,Male,Nicolas Chatelain,"13 January 1970 in Amiens, Somme (FRA)",France,6,181 cm / 70 kg,181.0,70,1970-01-13
6,Competed in Olympic Games,Male,Patrick Chila,"27 November 1969 in Ris-Orangis, Essonne (FRA)",France,7,180 cm / 73 kg,180.0,73,1969-11-27


In [305]:
df_nreduced.info()

<class 'pandas.core.frame.DataFrame'>
Index: 103028 entries, 1 to 145497
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Roles         103028 non-null  object        
 1   Sex           103028 non-null  object        
 2   Used name     103028 non-null  object        
 3   Born          102946 non-null  object        
 4   NOC           103028 non-null  object        
 5   athlete_id    103028 non-null  int64         
 6   Measurements  103028 non-null  object        
 7   height_cm     103028 non-null  float64       
 8   weight_kg     103028 non-null  object        
 9   born_date     102561 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(7)
memory usage: 8.6+ MB


In [306]:
# Parsing out country, region and city in `born` column
location_pattern = r'in ([\w\s-]+), ([\w\s-]+) \((\w+)\)'
df_nreduced[['born_city', 'born_region', 'born_country']] = df_nreduced['Born'].str.extract(location_pattern, expand=True)
df_nreduced

C:\Users\danie\AppData\Local\Temp\ipykernel_18304\42845723.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nreduced[['born_city', 'born_region', 'born_country']] = df_nreduced['Born'].str.extract(location_pattern, expand=True)
C:\Users\danie\AppData\Local\Temp\ipykernel_18304\42845723.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nreduced[['born_city', 'born_region', 'born_country']] = df_nreduced['Born'].str.extract(location_pattern, expand=True)
C:\Users\danie\AppData\Local\Temp\ipykernel_

,Roles,Sex,Used name,Born,NOC,athlete_id,Measurements,height_cm,weight_kg,born_date,born_city,born_region,born_country
1,Competed in Olympic Games,Male,Arnaud Boetsch,"1 April 1969 in Meulan, Yvelines (FRA)",France,2,183 cm / 76 kg,183.0,76,1969-04-01,Meulan,Yvelines,FRA
2,Competed in Olympic Games • Administrator,Male,Jean Borotra,"13 August 1898 in Biarritz, Pyrénées-Atlantiqu...",France,3,183 cm / 76 kg,183.0,76,1898-08-13,Biarritz,Pyrénées-Atlantiques,FRA
3,Competed in Olympic Games,Male,Jacques Brugnon,"11 May 1895 in Paris VIIIe, Paris (FRA)",France,4,168 cm / 64 kg,168.0,64,1895-05-11,Paris VIIIe,Paris,FRA
5,Competed in Olympic Games,Male,Nicolas Chatelain,"13 January 1970 in Amiens, Somme (FRA)",France,6,181 cm / 70 kg,181.0,70,1970-01-13,Amiens,Somme,FRA
6,Competed in Olympic Games,Male,Patrick Chila,"27 November 1969 in Ris-Orangis, Essonne (FRA)",France,7,180 cm / 73 kg,180.0,73,1969-11-27,Ris-Orangis,Essonne,FRA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
145473,Competed in Olympic Games,Male,Toms Andersons,"25 November 1993 in Rīga, Rīga (LAT)",Latvia,149200,185 cm / 86 kg,185.0,86,1993-11-25,Rīga,Rīga,LAT
145474,Competed in Olympic Games,Female,Nadine Hofstetter,"21 October 1994 in Romoos, Luzern (SUI)",Switzerland,149201,164 cm / 68 kg,164.0,68,1994-10-21,Romoos,Luzern,SUI
145495,Competed in Olympic Games,Female,Polina Luchnikova,"30 January 2002 in Serov, Sverdlovsk (RUS)",ROC,149222,167 cm / 61 kg,167.0,61,2002-01-30,Serov,Sverdlovsk,RUS
145496,Competed in Olympic Games,Female,Valeriya Merkusheva,"20 September 1999 in Moskva (Moscow), Moskva (...",ROC,149223,168 cm / 65 kg,168.0,65,1999-09-20,NaN,NaN,NaN


In [307]:
df_nreduced.value_counts('born_city', ascending=False).head(20)

born_city
Budapest          979
Sydney            399
Toronto           393
Seoul             371
Oslo              352
Melbourne         342
Buenos Aires      331
Montréal          330
Berlin            327
Rio de Janeiro    307
Barcelona         304
São Paulo         302
Helsinki          294
Sofia             284
London            279
Madrid            277
Minsk             266
New York          250
Stockholm         242
Vancouver         240
Name: count, dtype: int64

In [308]:
df_nreduced.columns

Index(['Roles', 'Sex', 'Used name', 'Born', 'NOC', 'athlete_id',
       'Measurements', 'height_cm', 'weight_kg', 'born_date', 'born_city',
       'born_region', 'born_country'],
      dtype='object')

In [309]:
columns_to_keep = ['athlete_id', 'Used name', 'Sex', 'born_date', 'born_city', 'born_region', 'born_country', 'NOC', 'height_cm', 'weight_kg']
df_clean = df_nreduced[columns_to_keep]
df_clean.head()

,athlete_id,Used name,Sex,born_date,born_city,born_region,born_country,NOC,height_cm,weight_kg
1,2,Arnaud Boetsch,Male,1969-04-01,Meulan,Yvelines,FRA,France,183.0,76
2,3,Jean Borotra,Male,1898-08-13,Biarritz,Pyrénées-Atlantiques,FRA,France,183.0,76
3,4,Jacques Brugnon,Male,1895-05-11,Paris VIIIe,Paris,FRA,France,168.0,64
5,6,Nicolas Chatelain,Male,1970-01-13,Amiens,Somme,FRA,France,181.0,70
6,7,Patrick Chila,Male,1969-11-27,Ris-Orangis,Essonne,FRA,France,180.0,73


In [310]:
df_clean.to_csv('./data_clean/bios_cleaned.csv', index=False)

In [ ]:
# Clean results file as an excercise